# Historgrams of temperature distribution

The first few plots will show the distribution of temperature in halos, in different ways:

- Gas mass fraction vs. temperature histogram for individual halos of different masses
- Gas mass fraction vs. temperature histogram for all halos within a certain mass bin, averaged
- Gas mass fraction vs. temperature histogram for all halos in the simulation, averaged

## Set-up

Setting up the Python env

In [38]:
# IMPORTS
import logging
import logging.config
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
import os
import sys
from numpy.typing import ArrayLike
from pathlib import Path
from typing import Any, Union

import illustris_python as il

In [39]:
# import the helper scripts
module_path = Path(os.getcwd()) / ".." / "src"
sys.path.append(str(module_path.resolve()))
sys.path.append(str(module_path.resolve() / "helpers"))
from helpers import logging_config, compute

ModuleNotFoundError: No module named 'constants'

Set up a small logging environment for impromptu logging during lengthy operations:

In [40]:
logging_cfg = logging_config.get_logging_config("INFO")
logging.config.dictConfig(logging_cfg)
logger = logging.getLogger("root")
# test setup
logger.info("I am a test log!")

INFO: I am a test log!


## Individual halos

We begin with plotting the temperature distribution of individual halos.